In [7]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import warnings
import matplotlib.colors as mcolors

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

def criar_dataframe_sumarizado(df):
    # Extract unique values
    razao_social = df['RAZAO SOCIAL'].tolist()
    responsavel = df['RESPONSAVEL'].tolist()
    municipio_uf = df['MUNICIPIO / UF'].tolist()
    populacao = df['POPULACAO'].tolist()
    contato_responsavel = df['CONTATO RESPONSAVEL'].tolist()
    contato_distribuidor = df['CONTATO DISTRIBUIDOR'].tolist()
    endereco = df['SEDE DISTRIBUIDOR'].tolist()

    # Find the maximum length among the lists
    max_length = max(len(razao_social), len(responsavel), len(municipio_uf), len(populacao), len(contato_responsavel), len(contato_distribuidor))

    # Pad lists with None to make them the same length
    razao_social.extend([None] * (max_length - len(razao_social)))
    responsavel.extend([None] * (max_length - len(responsavel)))
    municipio_uf.extend([None] * (max_length - len(municipio_uf)))
    populacao.extend([None] * (max_length - len(populacao)))
    contato_responsavel.extend([None] * (max_length - len(contato_responsavel)))
    contato_distribuidor.extend([None] * (max_length - len(contato_distribuidor)))
    endereco.extend([None] * (max_length - len(endereco)))

    # Create a new DataFrame
    summary_df = pd.DataFrame({
        'DISTRIBUIDOR': razao_social,
        'RESPONSAVEL': responsavel,
        'MUNICIPIO / UF': municipio_uf,
        'POPULACAO': populacao,
        'CONTATO RESPONSAVEL': contato_responsavel,
        'CONTATO DISTRIBUIDOR': contato_distribuidor,
        'SEDE DISTRIBUIDOR': endereco
    })

    # Replace NaN in 'RESPONSAVEL' column with 'EM BRANCO'
    summary_df.dropna(subset=['DISTRIBUIDOR'], inplace=True)

    # Replace NaN in 'RESPONSAVEL' column with 'EM BRANCO'
    summary_df['RESPONSAVEL'].fillna('SEM RESPONSAVEL', inplace=True)

    # Add a new column 'COR' with RGBA hexadecimal values for each 'RAZAO SOCIAL'
    unique_razao_social = summary_df['DISTRIBUIDOR'].unique()
    color_mapping = {razao: f"{hash(razao) % 0xffffff:06x}FF" for razao in unique_razao_social}
    summary_df['COR'] = summary_df['DISTRIBUIDOR'].map(color_mapping)

    return summary_df

def ler_arquivo_kml(file_path):
    """Lê arquivos KML em um diretório e retorna os dados relevantes."""
    df_data = []
    tree = ET.parse(file_path)
    root = tree.getroot()

    for placemark in root.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
        coordenadas = placemark.find('.//{http://www.opengis.net/kml/2.2}coordinates').text
        descricao = placemark.find('.//{http://www.opengis.net/kml/2.2}description').text
        coordenadas = ' '.join(coordenadas.split())

        df_data.append({'MUNICIPIO': name, 'DESCRICAO': descricao, 'COORDENADAS': coordenadas })
    return df_data

def processar_arquivos_kml(diretorio):
    """Processa todos os arquivos KML no diretório e retorna um DataFrame."""
    df_data = []
    
    for estado in os.listdir(diretorio):
        estado_path = os.path.join(diretorio, estado)

        for tipo in os.listdir(estado_path):
            tipo_path = os.path.join(estado_path, tipo)
            
            if 'kml' in tipo and os.path.isdir(tipo_path):
                for file_name in os.listdir(tipo_path):
                    if file_name.endswith('.kml'):
                        file_path = os.path.join(tipo_path, file_name)
                        dados = ler_arquivo_kml(file_path)
                        df_data.extend(dados)
    return df_data

def merge_arquivos(data_frame, file_path):
    df_distribuidores = criar_dataframe_sumarizado(data_frame)
    df_municipios_data = processar_arquivos_kml(file_path)
    
    df_municipios = pd.DataFrame(df_municipios_data)

    df_distribuidores.dropna(subset=['MUNICIPIO / UF'], inplace=True)

    df_merged = pd.merge(df_distribuidores, df_municipios, how='left', left_on='MUNICIPIO / UF', right_on='DESCRICAO')

    return df_merged

######################################################################################################    

def gerar_estilos(cor):
    return f'''
    <Style id="poly-{cor}-1200-77-nodesc-normal">
      <LineStyle>
        <color>{cor}</color>
        <width>2</width>
      </LineStyle>
      <PolyStyle>
        <color>{cor}</color>
        <fill>1</fill>
        <outline>1</outline>
      </PolyStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="poly-{cor}-1200-77-nodesc-highlight">
      <LineStyle>
        <color>{cor}</color>
        <width>3</width>
      </LineStyle>
      <PolyStyle>
        <color>{cor}</color>
        <fill>1</fill>
        <outline>1</outline>
      </PolyStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="poly-{cor}-1200-77-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#poly-{cor}-1200-77-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#poly-{cor}-1200-77-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
'''

def gerar_placemark(distribuidor, responsavel, endereco, municipio_uf, cor, coordenadas):
    return f'''
    <Placemark>
      <name>{municipio_uf}</name>
      <description>{distribuidor}, {responsavel}, {endereco}</description>
      <styleUrl>#poly-{cor}-1200-77-nodesc</styleUrl>
      <Polygon>
        <outerBoundaryIs>
          <LinearRing>
            <tessellate>1</tessellate>
            <coordinates>
              {coordenadas}
            </coordinates>
          </LinearRing>
        </outerBoundaryIs>
      </Polygon>
    </Placemark>
'''

def dados_template(dados_template, distributor):
    template = f'''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Document>
  <name>{distributor}</name>
'''

    cores_set = set()
    for municipio_data in dados_template.itertuples():
        cor = getattr(municipio_data, 'COR', None)
        if cor and cor not in cores_set:
            template += gerar_estilos(cor)
            cores_set.add(cor)

        regional = getattr(municipio_data, 'REGIONAL', None)
        distribuidor = getattr(municipio_data, 'DISTRIBUIDOR', None)
        responsavel = getattr(municipio_data, 'RESPONSAVEL', None)
        municipio = getattr(municipio_data, 'MUNICIPIO', None)
        municipio_uf = getattr(municipio_data, 'DESCRICAO', None)
        coordenadas = getattr(municipio_data, 'COORDENADAS', None)
        contato = getattr(municipio_data, 'CONTATO DISTRIBUIDOR', None)
        endereco = getattr(municipio_data, 'SEDE DISTRIBUIDOR', None)
        cor = getattr(municipio_data, 'COR', None)

        template += gerar_placemark(distribuidor, responsavel, endereco, municipio_uf, cor, coordenadas)

    template += '''
  </Document>
</kml>
'''
    return template




########################################################################################################################

def create_individual_templates(df_merged):
    unique_distributors = df_merged['DISTRIBUIDOR'].unique()

    output_directory = r'C:\Users\bruno\OneDrive\Área de Trabalho\Fluxo de trabalho\mapeamento\saida\distribuidores_kml\por_distribuidor'

    for distributor in unique_distributors:
      distributor_data = df_merged[df_merged['DISTRIBUIDOR'] == distributor]
      template = dados_template(distributor_data, distributor)
      # Save the template to a file in the specified directory
      template_file_path = os.path.join(output_directory, f"{distributor}.kml")
      with open(template_file_path, 'w', encoding='utf-8') as f:
        f.write(template)


def limpa_diretorio(diretorio):
    try:
        # Verifica se o diretório existe
        if os.path.exists(diretorio):
            # Lista todos os arquivos no diretório
            arquivos = os.listdir(diretorio)

            # Itera sobre os arquivos e remove cada um
            for arquivo in arquivos:
                caminho_completo = os.path.join(diretorio, arquivo)
                if os.path.isfile(caminho_completo):
                    os.remove(caminho_completo)
                elif os.path.isdir(caminho_completo):
                    # Se for um diretório, remove recursivamente
                    os.rmdir(caminho_completo)

            print(f"Limpeza concluída no diretório: {diretorio}")

        else:
            print(f"Diretório não encontrado: {diretorio}")

    except Exception as e:
        print(f"Ocorreu um erro durante a limpeza do diretório: {e}")

def main():
    
    diretorio_a_limpar = r'C:\Users\bruno\OneDrive\Área de Trabalho\Fluxo de trabalho\mapeamento\saida\distribuidores_kml\por_distribuidor'
    limpa_diretorio(diretorio_a_limpar)
    file_path = r'C:\Users\bruno\OneDrive\Área de Trabalho\Fluxo de trabalho\mapeamento\kml-brasil-master\lib\2010\municipios'
    df = pd.read_excel(r'C:\Users\bruno\OneDrive\Área de Trabalho\Fluxo de trabalho\mapeamento\excel\localidades.xlsx', sheet_name='BASE LIMPA')
    df_result = merge_arquivos(df, file_path)

    # Create individual templates for each distributor
    create_individual_templates(df_result)

if __name__ == "__main__":
    main()
